In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import FunctionTransformer
from sqlite3 import connect
from ydata_profiling import ProfileReport

In [2]:
from IPython.display import display, HTML
import warnings

In [3]:
pd.set_option('display.max_rows', 100)

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:.2f}".format

In [6]:
INPUT = 'przetwarzanie_faktur_zadanie.csv'

In [7]:
CONFIG = 'config.json'

In [8]:
# Get the absolute path of the file within the INPUTS directory
FILE_PATH = os.path.join(os.getcwd(), 'INPUTS', INPUT)

# Print the file path
print(FILE_PATH)

C:\Users\KonuTech\PycharmProjects\yet-another-analysis\INPUTS\przetwarzanie_faktur_zadanie.csv


In [9]:
# Get the absolute path of the config within the CONFIGS directory
CONFIG_PATH = os.path.join(os.getcwd(), 'CONFIGS', CONFIG)

# Print the fileconfig_pathpath
print(CONFIG_PATH)

C:\Users\KonuTech\PycharmProjects\yet-another-analysis\CONFIGS\config.json


In [10]:
with open(CONFIG_PATH, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [11]:
# INDEX_COL = CONFIG["INPUTS"]["INDEX_COLUMNS"]
SEP = CONFIG["INPUTS"]["SEPARATOR"]
# DECIMAL = CONFIG["INPUTS"]["DECIMAL"]
ENCODING = CONFIG["INPUTS"]["ENCODING"]
DATE_COLUMNS = CONFIG["INPUTS"]["DATE_COLUMNS"]
# FLOAT_PRECISION = CONFIG["INPUTS"]["FLOAT_PRECISION"]
DTYPE = CONFIG["INPUTS"]["DTYPE"]
COLUMNS_WITH_NAN_VALUES = CONFIG["INPUTS"]["COLUMNS_WITH_NAN_VALUES"]
# # X_ORDER = CONFIG["X_ORDER"]
HUE_ORDER = CONFIG["HUE_ORDER"]
COL_ORDER = CONFIG["COL_ORDER"]
WEEKDAY_ORDER = CONFIG["WEEKDAY_ORDER"]

In [12]:
# INDEX_COL

In [13]:
SEP

';'

In [14]:
ENCODING

'utf-8'

In [15]:
DATE_COLUMNS

['StartDateTimeUtc']

In [16]:
DTYPE

{'ID_Process': 'object',
 'ID_Document': 'object',
 'Queue': 'object',
 'ProcessingDuration': 'int64',
 'Duration': 'int64',
 'InputPageCount': 'int64',
 'MessagesInQueue': 'int64',
 'Retries': 'int64'}

In [17]:
COLUMNS_WITH_NAN_VALUES

['Duration']

In [18]:
HUE_ORDER

['potok_0-0',
 'potok_1-1',
 'potok_1-2',
 'potok_2-1',
 'potok_2-2',
 'potok_3-1',
 'potok_3-2']

In [19]:
COL_ORDER

['potok_0', 'potok_1', 'potok_2', 'potok_3']

In [20]:
WEEKDAY_ORDER

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [21]:
df = pd.read_csv(
    filepath_or_buffer=FILE_PATH,
#     index_col=INDEX_COL,
    sep=SEP,
    encoding=ENCODING,
    engine="c",
    low_memory=False,
    parse_dates=DATE_COLUMNS,
#     dtype=DTYPE
)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121190 entries, 0 to 1121189
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ID_Process          1121190 non-null  int64         
 1   Queue               1107824 non-null  object        
 2   ID_Document         1107824 non-null  float64       
 3   ProcessingDuration  1085406 non-null  float64       
 4   Duration            1080011 non-null  float64       
 5   InputPageCount      1085406 non-null  float64       
 6   MessagesInQueue     1121190 non-null  int64         
 7   Retries             1121190 non-null  int64         
 8   StartDateTimeUtc    1121190 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 77.0+ MB


In [23]:
df.head()

,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc
0,2299375,potok_1-1,2270386.00,2583.00,2940.00,1.00,4,0,2023-03-02 12:47:31.880
1,202658,potok_1-1,245850.00,5486.00,6626.00,2.00,6,0,2023-02-10 08:33:17.910
2,2323656,potok_3-1,2304772.00,2079.00,2606.00,1.00,7,0,2023-03-03 13:35:13.690
3,115622,potok_1-2,165259.00,2915.00,3228.00,2.00,1,0,2023-02-15 14:18:19.550
4,343261,potok_1-1,237985.00,2480.00,3114.00,1.00,9,0,2023-02-15 07:26:05.317


In [24]:
df.tail()

,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc
1121185,2627984,potok_3-2,2571026.00,3040.00,3683.00,1.00,2,0,2023-03-13 19:29:07.610
1121186,2492632,potok_3-1,2457336.00,3127.00,3425.00,1.00,6,0,2023-03-09 12:50:04.387
1121187,2771801,potok_3-2,2812898.00,298.00,715.00,2.00,0,0,2023-03-24 07:14:42.087
1121188,2343195,potok_1-2,2328046.00,9247.00,9689.00,1.00,4,0,2023-03-05 21:14:38.007
1121189,272829,potok_1-2,11148.00,74.00,10153.00,3.00,0,0,2023-02-13 10:47:07.257


In [25]:
df.describe()

,ID_Process,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries
count,1121190.00,1107824.00,1085406.00,1080011.00,1085406.00,1121190.00,1121190.00
mean,1623558.71,1616588.28,4187.53,25350.38,1.21,9.01,0.00
std,1131326.47,1122477.06,5195.52,323548.18,2.13,52.46,0.09
min,1.00,3.00,34.00,271.00,1.00,0.00,0.00
25%,280298.25,276786.75,2264.00,2714.00,1.00,1.00,0.00
50%,2311131.50,2295873.50,2973.00,3707.00,1.00,3.00,0.00
75%,2591428.75,2572823.25,4841.00,7064.00,1.00,7.00,0.00
max,2883900.00,2861842.00,1422420.00,45199914.00,736.00,2018.00,9.00


In [26]:
df.isnull().sum()

ID_Process                0
Queue                 13366
ID_Document           13366
ProcessingDuration    35784
Duration              41179
InputPageCount        35784
MessagesInQueue           0
Retries                   0
StartDateTimeUtc          0
dtype: int64

## preprocessing

## ON/OFF -- Filling NaN

In [27]:
# df['ProcessingDuration'].fillna(1e-1000, inplace=True)
# df['Duration'].fillna(1e-1000, inplace=True)
# df['InputPageCount'].fillna(0, inplace=True)

In [28]:
# Convert the time values to minutes, hours, and days
df['ProcessingDuration_sec'] = df['ProcessingDuration'] / 1000
# df['ProcessingDuration_min'] = df['ProcessingDuration'] / (1000 * 60)
# df['ProcessingDuration_hours'] = df['ProcessingDuration'] / (1000 * 60 * 60)
# df['ProcessingDuration_days'] = df['ProcessingDuration'] / (1000 * 60 * 60 * 24)

In [29]:
# Convert the time values to minutes, hours, and days
df['Duration_sec'] = df['Duration'] / 1000
# df['Duration_min'] = df['Duration'] / (1000 * 60)
# df['Duration_hours'] = df['Duration'] / (1000 * 60 * 60)
# df['Duration_days'] = df['Duration'] / (1000 * 60 * 60 * 24)

In [30]:
# Create a FunctionTransformer object with a logarithmic transformation function
# log_transformer = FunctionTransformer(np.log1p)
log_transformer = FunctionTransformer(np.log10)

# Apply the logarithmic transformation to the "Sales" column
df['Log_ProcessingDuration'] = log_transformer.transform(df[['ProcessingDuration']] + 1)
df['Log_ProcessingDuration_sec'] = log_transformer.transform(df[['ProcessingDuration_sec']] + 1)
# df['Log_ProcessingDuration_min'] = log_transformer.transform(df[['ProcessingDuration_min']] + 1)
# df['Log_ProcessingDuration_hours'] = log_transformer.transform(df[['ProcessingDuration_hours']] + 1)
# df['Log_ProcessingDuration_days'] = log_transformer.transform(df[['ProcessingDuration_days']] + 1)

df['Log_Duration'] = log_transformer.transform(df[['Duration']] + 1)
df['Log_Duration_sec'] = log_transformer.transform(df[['Duration_sec']] + 1)
# df['Log_Duration_min'] = log_transformer.transform(df[['Duration_min']] + 1)
# df['Log_Duration_hours'] = log_transformer.transform(df[['Duration_hours']] + 1)
# df['Log_Duration_days'] = log_transformer.transform(df[['Duration_days']] + 1)


log_transformer = FunctionTransformer(np.log10)
df['Log_InputPageCount'] = log_transformer.transform(df[['InputPageCount']] + 1)
# df['Log_Duration_sec'] = log_transformer.transform(df[['Duration_sec']] + 1)
# df['Log_Duration_min'] = log_transformer.transform(df[['Duration_min']] + 1)
# df['Log_Duration_hours'] = log_transformer.transform(df[['Duration_hours']] + 1)
# df['Log_Duration_days'] = log_transformer.transform(df[['Duration_days']] + 1)

log_transformer = FunctionTransformer(np.log10)
df['Log_MessagesInQueue'] = log_transformer.transform(df[['MessagesInQueue']] + 1)
# df['Log_Duration_sec'] = log_transformer.transform(df[['Duration_sec']] + 1)
# df['Log_Duration_min'] = log_transformer.transform(df[['Duration_min']] + 1)
# df['Log_Duration_hours'] = log_transformer.transform(df[['Duration_hours']] + 1)
# df['Log_Duration_days'] = log_transformer.transform(df[['Duration_days']] + 1)

In [31]:
df['Queue'] = df['Queue'].fillna('Potok_0-0')

In [32]:
df[['potok']] = 'potok_' + df['Queue'].str.extract(r'^potok_(\d+)')

In [33]:
for i, v in enumerate(sorted(df["Queue"].unique())):
    print("POTOK:", i, v)
    print(df[df["Queue"] == v].isnull().sum(), "\n")

POTOK: 0 Potok_0-0
ID_Process                        0
Queue                             0
ID_Document                   13366
ProcessingDuration                0
Duration                        530
InputPageCount                    0
MessagesInQueue                   0
Retries                           0
StartDateTimeUtc                  0
ProcessingDuration_sec            0
Duration_sec                    530
Log_ProcessingDuration            0
Log_ProcessingDuration_sec        0
Log_Duration                    530
Log_Duration_sec                530
Log_InputPageCount                0
Log_MessagesInQueue               0
potok                         13366
dtype: int64 

POTOK: 1 potok_1-1
ID_Process                        0
Queue                             0
ID_Document                       0
ProcessingDuration            10562
Duration                      11557
InputPageCount                10562
MessagesInQueue                   0
Retries                           0
StartDateTi

In [34]:
# Convert to datetime
df['date_time'] = pd.to_datetime(df['StartDateTimeUtc'])

# Extract date-related columns
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['week'] = df['date_time'].dt.week
df['day'] = df['date_time'].dt.day
df['hour'] = df['date_time'].dt.hour
df['minute'] = df['date_time'].dt.minute
df['second'] = df['date_time'].dt.second
df['microsecond'] = df['date_time'].dt.microsecond
# df['weekday'] = df['date_time'].dt.weekday
# Create a new column indicating the weekday (Monday is 1st, Sunday is 7th)
df['weekday'] = df['date_time'].dt.weekday.apply(lambda x: x + 1 if x != 6 else 7)
df['weekday_name'] = df['date_time'].dt.strftime('%A')
df['month_name'] = df['date_time'].dt.strftime('%B')

# Create flag for typical working hours (9 AM to 5 PM on weekdays)
df['working_hours'] = np.where((df['weekday'].isin([1,2,3,4,5])) & (df['hour'] >= 9) & (df['hour'] < 17), 1, 0)

In [35]:
df[df['weekday'].isin([6,7])].head()

,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
19,2347610,potok_1-2,2323521.00,5174.00,5681.00,1.00,1,0,2023-03-05 11:53:10.883,5.17,5.68,3.71,0.79,3.75,0.82,0.30,0.30,potok_1,2023-03-05 11:53:10.883,2023,3,9,5,11,53,10,883000,7,Sunday,March,0
46,303874,potok_1-1,13020.00,2331.00,2845.00,1.00,4,0,2023-02-19 15:10:10.867,2.33,2.85,3.37,0.52,3.45,0.58,0.30,0.70,potok_1,2023-02-19 15:10:10.867,2023,2,7,19,15,10,10,867000,7,Sunday,February,0
48,785128,potok_3-2,693766.00,6149.00,7274.00,1.00,10,0,2023-02-05 19:47:58.283,6.15,7.27,3.79,0.85,3.86,0.92,0.30,1.04,potok_3,2023-02-05 19:47:58.283,2023,2,5,5,19,47,58,283000,7,Sunday,February,0
62,129788,potok_1-2,6491.00,10984.00,44663.00,1.00,19,0,2023-02-19 03:27:32.237,10.98,44.66,4.04,1.08,4.65,1.66,0.30,1.30,potok_1,2023-02-19 03:27:32.237,2023,2,7,19,3,27,32,237000,7,Sunday,February,0
85,2791819,potok_3-1,2683288.00,4195.00,4518.00,1.00,1,0,2023-03-19 20:20:47.470,4.20,4.52,3.62,0.72,3.66,0.74,0.30,0.30,potok_3,2023-03-19 20:20:47.470,2023,3,11,19,20,20,47,470000,7,Sunday,March,0


## samplowanie

In [36]:
df_sample = df.sample(frac=0.01, replace=False)
df_sample

,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
921071,2502692,potok_3-2,2576944.00,2554.00,2968.00,1.00,0,0,2023-03-14 00:27:45.007,2.55,2.97,3.41,0.55,3.47,0.60,0.30,0.00,potok_3,2023-03-14 00:27:45.007,2023,3,11,14,0,27,45,7000,2,Tuesday,March,0
728668,2326501,potok_1-2,2302869.00,7329.00,8075.00,1.00,17,0,2023-03-03 13:03:30.447,7.33,8.07,3.87,0.92,3.91,0.96,0.30,1.26,potok_1,2023-03-03 13:03:30.447,2023,3,9,3,13,3,30,447000,5,Friday,March,1
470448,2253412,potok_1-2,2223581.00,3143.00,3535.00,1.00,0,0,2023-03-01 13:52:10.230,3.14,3.54,3.50,0.62,3.55,0.66,0.30,0.00,potok_1,2023-03-01 13:52:10.230,2023,3,9,1,13,52,10,230000,3,Wednesday,March,1
843769,2399922,potok_1-2,2403696.00,523.00,933.00,1.00,1,0,2023-03-08 14:34:17.580,0.52,0.93,2.72,0.18,2.97,0.29,0.30,0.30,potok_1,2023-03-08 14:34:17.580,2023,3,10,8,14,34,17,580000,3,Wednesday,March,1
692528,2340114,potok_1-2,2328029.00,1291.00,1783.00,1.00,6,0,2023-03-05 20:46:50.370,1.29,1.78,3.11,0.36,3.25,0.44,0.30,0.85,potok_1,2023-03-05 20:46:50.370,2023,3,9,5,20,46,50,370000,7,Sunday,March,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110008,2334024,potok_3-2,2318644.00,10453.00,13702.00,2.00,11,0,2023-03-04 16:07:12.770,10.45,13.70,4.02,1.06,4.14,1.17,0.48,1.08,potok_3,2023-03-04 16:07:12.770,2023,3,9,4,16,7,12,770000,6,Saturday,March,0
122086,2479452,potok_1-2,2482428.00,2620.00,3045.00,1.00,0,0,2023-03-10 17:04:40.647,2.62,3.04,3.42,0.56,3.48,0.61,0.30,0.00,potok_1,2023-03-10 17:04:40.647,2023,3,10,10,17,4,40,647000,5,Friday,March,0
115549,179110,potok_1-2,192070.00,71.00,432.00,1.00,2,0,2023-02-07 11:25:45.560,0.07,0.43,1.86,0.03,2.64,0.16,0.30,0.48,potok_1,2023-02-07 11:25:45.560,2023,2,6,7,11,25,45,560000,2,Tuesday,February,1
507923,136628,potok_1-2,214894.00,99.00,344808.00,1.00,2,0,2023-02-07 20:15:41.637,0.10,344.81,2.00,0.04,5.54,2.54,0.30,0.48,potok_1,2023-02-07 20:15:41.637,2023,2,6,7,20,15,41,637000,2,Tuesday,February,0


In [37]:
df_sample.describe()

,ID_Process,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,year,month,week,day,hour,minute,second,microsecond,weekday,working_hours
count,11212.00,11084.00,10870.00,10802.00,10870.00,11212.00,11212.00,10870.00,10802.00,10870.00,10870.00,10802.00,10802.00,10870.00,11212.00,11212.00,11212.00,11212.00,11212.00,11212.00,11212.00,11212.00,11212.00,11212.00,11212.00
mean,1628766.74,1624068.73,4167.92,20776.29,1.22,8.63,0.00,4.17,20.78,3.43,0.63,3.65,0.77,0.33,0.61,2023.00,2.54,8.85,13.10,12.00,29.86,29.70,501288.71,3.30,0.60
std,1131286.30,1120409.40,4042.43,233006.85,2.29,40.76,0.06,4.04,233.01,0.50,0.27,0.49,0.40,0.09,0.49,0.00,0.50,2.32,7.42,4.02,17.24,17.19,290024.01,1.73,0.49
min,95.00,111.00,36.00,296.00,1.00,0.00,0.00,0.04,0.30,1.57,0.02,2.47,0.11,0.30,0.00,2023.00,2.00,5.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
25%,275335.75,277237.50,2278.25,2728.00,1.00,1.00,0.00,2.28,2.73,3.36,0.52,3.44,0.57,0.30,0.30,2023.00,2.00,7.00,7.00,9.00,15.00,15.00,250000.00,2.00,0.00
50%,2313305.50,2297591.00,2968.00,3707.00,1.00,3.00,0.00,2.97,3.71,3.47,0.60,3.57,0.67,0.30,0.60,2023.00,3.00,9.00,13.00,11.00,30.00,30.00,503000.00,3.00,1.00
75%,2590491.50,2574414.25,4899.50,7166.75,1.00,7.00,0.00,4.90,7.17,3.69,0.77,3.86,0.91,0.30,0.90,2023.00,3.00,11.00,17.00,14.00,45.00,45.00,753000.00,5.00,1.00
max,2883519.00,2861751.00,80377.00,7838873.00,220.00,1905.00,2.00,80.38,7838.87,4.91,1.91,6.89,3.89,2.34,3.28,2023.00,3.00,13.00,31.00,23.00,59.00,59.00,997000.00,7.00,1.00


In [38]:
duplicates = df[df.duplicated()]
print(duplicates)


Empty DataFrame
Columns: [ID_Process, Queue, ID_Document, ProcessingDuration, Duration, InputPageCount, MessagesInQueue, Retries, StartDateTimeUtc, ProcessingDuration_sec, Duration_sec, Log_ProcessingDuration, Log_ProcessingDuration_sec, Log_Duration, Log_Duration_sec, Log_InputPageCount, Log_MessagesInQueue, potok, date_time, year, month, week, day, hour, minute, second, microsecond, weekday, weekday_name, month_name, working_hours]
Index: []


In [39]:
print(df['ID_Process'].duplicated().sum())
print(df['ID_Document'].duplicated().sum())

0
13571


In [40]:
conn = connect(':memory:')
df.to_sql('data', conn)

1121190

In [41]:
select_01 = pd.read_sql(
"""
    SELECT DISTINCT
        StartDateTimeUtc
        ,ID_Process
        ,ID_Document
    FROM
        data
    ORDER BY
        StartDateTimeUtc
        ,ID_Process
        ,ID_Document
    
""",
conn
)

In [42]:
select_01

,StartDateTimeUtc,ID_Process,ID_Document
0,2023-02-01 00:06:11.107000,639283,621337.00
1,2023-02-01 00:06:33.830000,548712,646489.00
2,2023-02-01 00:11:00.453000,375853,554847.00
3,2023-02-01 00:11:00.650000,364211,810000.00
4,2023-02-01 00:11:00.847000,466012,789310.00
...,...,...,...
1121185,2023-03-31 23:52:57.150000,2793101,2705518.00
1121186,2023-03-31 23:53:25.927000,2840520,2706703.00
1121187,2023-03-31 23:53:26.757000,2743254,2709756.00
1121188,2023-03-31 23:53:27.237000,2795479,2746911.00


In [43]:
select_02 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    StartDateTimeUtc
                    ,COUNT(StartDateTimeUtc) AS StartDateTimeUtc_COUNT
                FROM
                    data
                GROUP BY
                    StartDateTimeUtc
            )
    
    SELECT
        StartDateTimeUtc
        ,StartDateTimeUtc_COUNT
    FROM
        count
    WHERE
        StartDateTimeUtc_COUNT > 1

""",
conn
)

In [44]:
select_02

,StartDateTimeUtc,StartDateTimeUtc_COUNT
0,2023-02-01 00:24:12.730000,2
1,2023-02-01 00:24:13.037000,2
2,2023-02-01 00:58:02.987000,2
3,2023-02-01 00:58:03.087000,2
4,2023-02-01 00:58:03.823000,2
...,...,...
12376,2023-03-31 19:19:25.090000,2
12377,2023-03-31 19:19:27.193000,2
12378,2023-03-31 19:51:27.397000,2
12379,2023-03-31 22:49:07.967000,2


In [45]:
select_03 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    StartDateTimeUtc
                    ,COUNT(StartDateTimeUtc) AS StartDateTimeUtc_COUNT
                FROM
                    data
                GROUP BY
                    StartDateTimeUtc
            )
    
    SELECT
        *
    FROM
        data
    WHERE
        StartDateTimeUtc IN (
            SELECT DISTINCT 
                StartDateTimeUtc
            FROM
                count
            WHERE
                StartDateTimeUtc_COUNT > 1
        )
    ORDER BY
        StartDateTimeUtc

""",
conn
)

In [46]:
select_03

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,22515,793991,potok_1-2,706616.00,9086.00,37984.00,1.00,2,0,2023-02-01 00:24:12.730000,9.09,37.98,3.96,1.00,4.58,1.59,0.30,0.48,potok_1,2023-02-01 00:24:12.730000,2023,2,5,1,0,24,12,730000,3,Wednesday,February,0
1,45533,409858,potok_1-2,676908.00,8156.00,37068.00,1.00,7,0,2023-02-01 00:24:12.730000,8.16,37.07,3.91,0.96,4.57,1.58,0.30,0.90,potok_1,2023-02-01 00:24:12.730000,2023,2,5,1,0,24,12,730000,3,Wednesday,February,0
2,473334,552830,potok_1-2,736114.00,1518.00,29316.00,1.00,2,0,2023-02-01 00:24:13.037000,1.52,29.32,3.18,0.40,4.47,1.48,0.30,0.48,potok_1,2023-02-01 00:24:13.037000,2023,2,5,1,0,24,13,37000,3,Wednesday,February,0
3,1083241,649766,potok_1-2,440904.00,3460.00,27615.00,1.00,0,0,2023-02-01 00:24:13.037000,3.46,27.61,3.54,0.65,4.44,1.46,0.30,0.00,potok_1,2023-02-01 00:24:13.037000,2023,2,5,1,0,24,13,37000,3,Wednesday,February,0
4,357837,709066,potok_1-2,444088.00,11193.00,38137.00,1.00,51,0,2023-02-01 00:58:02.987000,11.19,38.14,4.05,1.09,4.58,1.59,0.30,1.72,potok_1,2023-02-01 00:58:02.987000,2023,2,5,1,0,58,2,987000,3,Wednesday,February,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25799,654923,2850394,potok_3-2,2701881.00,6003.00,8030.00,1.00,21,0,2023-03-31 19:51:27.397000,6.00,8.03,3.78,0.85,3.90,0.96,0.30,1.34,potok_3,2023-03-31 19:51:27.397000,2023,3,13,31,19,51,27,397000,5,Friday,March,0
25800,90301,2707398,potok_3-2,2798543.00,4353.00,23724.00,1.00,0,0,2023-03-31 22:49:07.967000,4.35,23.72,3.64,0.73,4.38,1.39,0.30,0.00,potok_3,2023-03-31 22:49:07.967000,2023,3,13,31,22,49,7,967000,5,Friday,March,0
25801,587088,2732511,potok_3-2,2859349.00,3877.00,22478.00,1.00,1,0,2023-03-31 22:49:07.967000,3.88,22.48,3.59,0.69,4.35,1.37,0.30,0.30,potok_3,2023-03-31 22:49:07.967000,2023,3,13,31,22,49,7,967000,5,Friday,March,0
25802,443807,2701720,potok_3-2,2781951.00,14228.00,40655.00,3.00,7,0,2023-03-31 23:23:35.057000,14.23,40.66,4.15,1.18,4.61,1.62,0.60,0.90,potok_3,2023-03-31 23:23:35.057000,2023,3,13,31,23,23,35,57000,5,Friday,March,0


In [47]:
select_04 = pd.read_sql(
"""
                SELECT
                    ID_Document
                    ,COUNT(ID_Document) AS ID_Document_COUNT
                FROM
                    data
                GROUP BY
                    ID_Document
HAVING ID_Document_COUNT > 1

""",
conn
)

In [48]:
select_04

,ID_Document,ID_Document_COUNT
0,370.00,7
1,2775.00,2
2,3569.00,7
3,4948.00,7
4,5134.00,5
5,6722.00,5
6,7339.00,5
7,7968.00,7
8,9779.00,7
9,10741.00,5


In [49]:
select_05 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    ID_Document
                    ,COUNT(ID_Document) AS ID_Document_COUNT
                FROM
                    data
                GROUP BY
                    ID_Document
            )
    
    SELECT
        *
    FROM
        data
    WHERE
        ID_Document IN (
            SELECT DISTINCT 
                ID_Document
            FROM
                count
            WHERE
                ID_Document_COUNT > 1
        )
    ORDER BY
        ID_Document,
        StartDateTimeUtc

""",
conn
)

In [50]:
select_05.tail(100)

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
179,140602,223970,potok_2-2,37980.00,2808.00,3163.00,1.00,1,0,2023-02-13 00:33:07.120000,2.81,3.16,3.45,0.58,3.50,0.62,0.30,0.30,potok_2,2023-02-13 00:33:07.120000,2023,2,7,13,0,33,7,120000,1,Monday,February,0
180,745090,134579,potok_2-2,37980.00,4124.00,44387.00,1.00,1,0,2023-02-15 22:43:31.140000,4.12,44.39,3.62,0.71,4.65,1.66,0.30,0.30,potok_2,2023-02-15 22:43:31.140000,2023,2,7,15,22,43,31,140000,3,Wednesday,February,0
181,373656,119332,potok_2-2,37980.00,4063.00,42101.00,1.00,1,0,2023-02-15 22:58:03.367000,4.06,42.10,3.61,0.70,4.62,1.63,0.30,0.30,potok_2,2023-02-15 22:58:03.367000,2023,2,7,15,22,58,3,367000,3,Wednesday,February,0
182,16093,634809,potok_3-1,38968.00,1780.00,6023.00,1.00,0,0,2023-02-03 22:51:09.857000,1.78,6.02,3.25,0.44,3.78,0.85,0.30,0.00,potok_3,2023-02-03 22:51:09.857000,2023,2,5,3,22,51,9,857000,5,Friday,February,0
183,801387,73199,potok_3-1,38968.00,2931.00,3650.00,1.00,0,0,2023-02-13 00:22:43.413000,2.93,3.65,3.47,0.59,3.56,0.67,0.30,0.00,potok_3,2023-02-13 00:22:43.413000,2023,2,7,13,0,22,43,413000,1,Monday,February,0
184,307518,273711,potok_3-1,38968.00,2436.00,3323.00,1.00,0,0,2023-02-13 00:30:05.830000,2.44,3.32,3.39,0.54,3.52,0.64,0.30,0.00,potok_3,2023-02-13 00:30:05.830000,2023,2,7,13,0,30,5,830000,1,Monday,February,0
185,589807,150443,potok_3-1,38968.00,2122.00,2600.00,1.00,0,0,2023-02-13 00:33:06.383000,2.12,2.60,3.33,0.49,3.42,0.56,0.30,0.00,potok_3,2023-02-13 00:33:06.383000,2023,2,7,13,0,33,6,383000,1,Monday,February,0
186,647593,240193,potok_3-1,38968.00,2996.00,3813.00,1.00,0,0,2023-02-15 22:43:30.367000,3.00,3.81,3.48,0.60,3.58,0.68,0.30,0.00,potok_3,2023-02-15 22:43:30.367000,2023,2,7,15,22,43,30,367000,3,Wednesday,February,0
187,232731,315399,potok_3-1,38968.00,2941.00,3596.00,1.00,0,0,2023-02-15 22:58:02.623000,2.94,3.60,3.47,0.60,3.56,0.66,0.30,0.00,potok_3,2023-02-15 22:58:02.623000,2023,2,7,15,22,58,2,623000,3,Wednesday,February,0
188,1000953,153859,potok_3-1,38968.00,2710.00,4088.00,1.00,0,0,2023-02-15 23:12:04.047000,2.71,4.09,3.43,0.57,3.61,0.71,0.30,0.00,potok_3,2023-02-15 23:12:04.047000,2023,2,7,15,23,12,4,47000,3,Wednesday,February,0


## ID_Process is Unique

In [51]:
select_06 = pd.read_sql(
"""
                SELECT
                    ID_Process
                    ,COUNT(ID_Process) AS ID_Process_COUNT
                FROM
                    data
                GROUP BY
                    ID_Process
HAVING ID_Process_COUNT > 1

""",
conn
)

In [52]:
select_06

,ID_Process,ID_Process_COUNT


In [53]:
select_07 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    ID_Process
                    ,COUNT(ID_Process) AS ID_Process_COUNT
                FROM
                    data
                GROUP BY
                    ID_Process
            )
    
    SELECT
        *
    FROM
        data
    WHERE
        ID_Process IN (
            SELECT DISTINCT 
                ID_Process
            FROM
                count
            WHERE
                ID_Process_COUNT > 1
        )
    ORDER BY
        ID_Process

""",
conn
)

In [54]:
select_07

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours


## 370

In [55]:
select_08 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    ID_Document
                    ,COUNT(ID_Document) AS ID_Document_COUNT
                FROM
                    data
                WHERE
                 ID_Document = 370.00
                GROUP BY
                    ID_Document
            )
    
    SELECT
        *
    FROM
        data
    WHERE
        ID_Document IN (
            SELECT DISTINCT 
                ID_Document
            FROM
                count
            WHERE
                ID_Document_COUNT > 1
        )
    ORDER BY
        ID_Document,
        StartDateTimeUtc

""",
conn
)

In [56]:
select_08

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,407041,492356,potok_3-2,370.00,4160.00,29939.00,1.00,2,0,2023-02-03 22:58:03.620000,4.16,29.94,3.62,0.71,4.48,1.49,0.30,0.48,potok_3,2023-02-03 22:58:03.620000,2023,2,5,3,22,58,3,620000,5,Friday,February,0
1,145232,200388,potok_3-2,370.00,3164.00,3602.00,1.00,3,0,2023-02-13 00:22:46.007000,3.16,3.60,3.50,0.62,3.56,0.66,0.30,0.60,potok_3,2023-02-13 00:22:46.007000,2023,2,7,13,0,22,46,7000,1,Monday,February,0
2,999926,210746,potok_3-2,370.00,4320.00,29437.00,1.00,3,0,2023-02-13 00:30:08.300000,4.32,29.44,3.64,0.73,4.47,1.48,0.30,0.60,potok_3,2023-02-13 00:30:08.300000,2023,2,7,13,0,30,8,300000,1,Monday,February,0
3,476233,313448,potok_3-2,370.00,2324.00,3090.00,1.00,2,0,2023-02-13 00:33:08.840000,2.32,3.09,3.37,0.52,3.49,0.61,0.30,0.48,potok_3,2023-02-13 00:33:08.840000,2023,2,7,13,0,33,8,840000,1,Monday,February,0
4,372481,228319,potok_3-2,370.00,4617.00,130979.00,1.00,3,0,2023-02-15 22:43:34.460000,4.62,130.98,3.66,0.75,5.12,2.12,0.30,0.60,potok_3,2023-02-15 22:43:34.460000,2023,2,7,15,22,43,34,460000,3,Wednesday,February,0
5,187320,28579,potok_3-2,370.00,3500.00,3821.00,1.00,3,0,2023-02-15 22:58:05.773000,3.50,3.82,3.54,0.65,3.58,0.68,0.30,0.60,potok_3,2023-02-15 22:58:05.773000,2023,2,7,15,22,58,5,773000,3,Wednesday,February,0
6,933490,208746,potok_3-2,370.00,4506.00,26841.00,1.00,3,0,2023-02-15 23:12:05.880000,4.51,26.84,3.65,0.74,4.43,1.44,0.30,0.60,potok_3,2023-02-15 23:12:05.880000,2023,2,7,15,23,12,5,880000,3,Wednesday,February,0


## 2822388.00

In [57]:
select_08 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    ID_Document
                    ,COUNT(ID_Document) AS ID_Document_COUNT
                FROM
                    data
                WHERE
                 ID_Document = 2822388.00
                GROUP BY
                    ID_Document
            )
    
    SELECT
        *
    FROM
        data
    WHERE
        ID_Document IN (
            SELECT DISTINCT 
                ID_Document
            FROM
                count
            WHERE
                ID_Document_COUNT > 1
        )
    ORDER BY
        ID_Document,
        StartDateTimeUtc

""",
conn
)

In [58]:
select_08

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,100405,2754855,potok_3-2,2822388.00,87.00,591237.00,1.00,58,0,2023-03-23 04:56:44.767000,0.09,591.24,1.94,0.04,5.77,2.77,0.30,1.77,potok_3,2023-03-23 04:56:44.767000,2023,3,12,23,4,56,44,767000,4,Thursday,March,0
1,665054,2827268,potok_3-2,2822388.00,5589.00,338494.00,1.00,0,0,2023-03-23 04:56:44.767000,5.59,338.49,3.75,0.82,5.53,2.53,0.30,0.00,potok_3,2023-03-23 04:56:44.767000,2023,3,12,23,4,56,44,767000,4,Thursday,March,0
2,1078295,2765062,potok_3-2,2822388.00,82.00,388775.00,1.00,49,0,2023-03-23 04:56:44.767000,0.08,388.77,1.92,0.03,5.59,2.59,0.30,1.70,potok_3,2023-03-23 04:56:44.767000,2023,3,12,23,4,56,44,767000,4,Thursday,March,0


## 37980.00

In [59]:
select_09 = pd.read_sql(
"""
    WITH
    
    count AS (
                SELECT
                    ID_Document
                    ,COUNT(ID_Document) AS ID_Document_COUNT
                FROM
                    data
                WHERE
                 ID_Document = 37980.00
                GROUP BY
                    ID_Document
            )
    
    SELECT
        *
    FROM
        data
    WHERE
        ID_Document IN (
            SELECT DISTINCT 
                ID_Document
            FROM
                count
            WHERE
                ID_Document_COUNT > 1
        )
    ORDER BY
        ID_Document,
        StartDateTimeUtc

""",
conn
)

In [60]:
select_09

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,74580,321115,potok_2-2,37980.00,4941.00,37554.00,1.00,1,0,2023-02-13 00:22:44.150000,4.94,37.55,3.69,0.77,4.57,1.59,0.30,0.30,potok_2,2023-02-13 00:22:44.150000,2023,2,7,13,0,22,44,150000,1,Monday,February,0
1,440416,316547,potok_2-2,37980.00,3752.00,38643.00,1.00,1,0,2023-02-13 00:30:06.503000,3.75,38.64,3.57,0.68,4.59,1.60,0.30,0.30,potok_2,2023-02-13 00:30:06.503000,2023,2,7,13,0,30,6,503000,1,Monday,February,0
2,140602,223970,potok_2-2,37980.00,2808.00,3163.00,1.00,1,0,2023-02-13 00:33:07.120000,2.81,3.16,3.45,0.58,3.50,0.62,0.30,0.30,potok_2,2023-02-13 00:33:07.120000,2023,2,7,13,0,33,7,120000,1,Monday,February,0
3,745090,134579,potok_2-2,37980.00,4124.00,44387.00,1.00,1,0,2023-02-15 22:43:31.140000,4.12,44.39,3.62,0.71,4.65,1.66,0.30,0.30,potok_2,2023-02-15 22:43:31.140000,2023,2,7,15,22,43,31,140000,3,Wednesday,February,0
4,373656,119332,potok_2-2,37980.00,4063.00,42101.00,1.00,1,0,2023-02-15 22:58:03.367000,4.06,42.10,3.61,0.70,4.62,1.63,0.30,0.30,potok_2,2023-02-15 22:58:03.367000,2023,2,7,15,22,58,3,367000,3,Wednesday,February,0


### Ile jest roznych 'Queue' dla tego samego 'ID_Document'? 

In [61]:
select_10 = pd.read_sql(
"""
    WITH
    
    dist AS (
                SELECT DISTINCT
                     ID_Document
                    ,Queue
                FROM
                    data
            )
            
    
    SELECT
        *
    FROM
        dist
    ORDER BY
        ID_Document
        ,Queue

""",
conn
)

In [62]:
select_10

,ID_Document,Queue
0,NaN,Potok_0-0
1,3.00,potok_1-1
2,4.00,potok_1-1
3,5.00,potok_1-2
4,6.00,potok_1-2
...,...,...
1107614,2861837.00,potok_3-2
1107615,2861838.00,potok_3-1
1107616,2861839.00,potok_3-1
1107617,2861840.00,potok_3-2


In [63]:
select_12 = pd.read_sql(
"""
    SELECT
         ID_Document
        ,COUNT(DISTINCT ID_Process) AS num_distinct_ID_Process
    FROM
        data
    GROUP BY
        ID_Document
    HAVING
        num_distinct_ID_Process > 1
    

""",
conn
)

In [64]:
select_12

,ID_Document,num_distinct_ID_Process
0,NaN,13366
1,370.00,7
2,2775.00,2
3,3569.00,7
4,4948.00,7
5,5134.00,5
6,6722.00,5
7,7339.00,5
8,7968.00,7
9,9779.00,7


## Czy uzyc sredniej, gdy ID_Document == Nan ?

In [65]:
select_13 = pd.read_sql(
"""
    SELECT
         ID_Document
        ,COUNT(DISTINCT ID_Process) AS ID_Process_COUNT
        ,SUM(InputPageCount) AS InputPageCount_SUM
        ,AVG(InputPageCount) AS InputPageCount_AVG
        ,SUM(ProcessingDuration) AS ProcessingDuration_SUM
        ,SUM(Duration) AS Duration_SUM
        ,AVG(ProcessingDuration) AS ProcessingDuration_AVG
        ,AVG(Duration) AS Duration_AVG
    FROM
        data
    GROUP BY
        ID_Document
    HAVING
        ID_Process_COUNT > 1
    

""",
conn
)

In [66]:
select_13

,ID_Document,ID_Process_COUNT,InputPageCount_SUM,InputPageCount_AVG,ProcessingDuration_SUM,Duration_SUM,ProcessingDuration_AVG,Duration_AVG
0,NaN,13366,15370.00,1.15,18306555.00,110834331.00,1369.64,8634.65
1,370.00,7,7.00,1.00,26591.00,227709.00,3798.71,32529.86
2,2775.00,2,2.00,1.00,15060.00,17577.00,7530.00,8788.50
3,3569.00,7,7.00,1.00,17713.00,24321.00,2530.43,3474.43
4,4948.00,7,7.00,1.00,25444.00,116880.00,3634.86,16697.14
5,5134.00,5,4.00,1.00,15886.00,184870.00,3971.50,36974.00
6,6722.00,5,3.00,1.00,7491.00,354801.00,2497.00,118267.00
7,7339.00,5,5.00,1.00,16556.00,137559.00,3311.20,27511.80
8,7968.00,7,7.00,1.00,16649.00,37927.00,2378.43,5418.14
9,9779.00,7,7.00,1.00,14385.00,20916.00,2055.00,2988.00


In [67]:
df

,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,2299375,potok_1-1,2270386.00,2583.00,2940.00,1.00,4,0,2023-03-02 12:47:31.880,2.58,2.94,3.41,0.55,3.47,0.60,0.30,0.70,potok_1,2023-03-02 12:47:31.880,2023,3,9,2,12,47,31,880000,4,Thursday,March,1
1,202658,potok_1-1,245850.00,5486.00,6626.00,2.00,6,0,2023-02-10 08:33:17.910,5.49,6.63,3.74,0.81,3.82,0.88,0.48,0.85,potok_1,2023-02-10 08:33:17.910,2023,2,6,10,8,33,17,910000,5,Friday,February,0
2,2323656,potok_3-1,2304772.00,2079.00,2606.00,1.00,7,0,2023-03-03 13:35:13.690,2.08,2.61,3.32,0.49,3.42,0.56,0.30,0.90,potok_3,2023-03-03 13:35:13.690,2023,3,9,3,13,35,13,690000,5,Friday,March,1
3,115622,potok_1-2,165259.00,2915.00,3228.00,2.00,1,0,2023-02-15 14:18:19.550,2.92,3.23,3.46,0.59,3.51,0.63,0.48,0.30,potok_1,2023-02-15 14:18:19.550,2023,2,7,15,14,18,19,550000,3,Wednesday,February,1
4,343261,potok_1-1,237985.00,2480.00,3114.00,1.00,9,0,2023-02-15 07:26:05.317,2.48,3.11,3.39,0.54,3.49,0.61,0.30,1.00,potok_1,2023-02-15 07:26:05.317,2023,2,7,15,7,26,5,317000,3,Wednesday,February,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121185,2627984,potok_3-2,2571026.00,3040.00,3683.00,1.00,2,0,2023-03-13 19:29:07.610,3.04,3.68,3.48,0.61,3.57,0.67,0.30,0.48,potok_3,2023-03-13 19:29:07.610,2023,3,11,13,19,29,7,610000,1,Monday,March,0
1121186,2492632,potok_3-1,2457336.00,3127.00,3425.00,1.00,6,0,2023-03-09 12:50:04.387,3.13,3.42,3.50,0.62,3.53,0.65,0.30,0.85,potok_3,2023-03-09 12:50:04.387,2023,3,10,9,12,50,4,387000,4,Thursday,March,1
1121187,2771801,potok_3-2,2812898.00,298.00,715.00,2.00,0,0,2023-03-24 07:14:42.087,0.30,0.71,2.48,0.11,2.85,0.23,0.48,0.00,potok_3,2023-03-24 07:14:42.087,2023,3,12,24,7,14,42,87000,5,Friday,March,0
1121188,2343195,potok_1-2,2328046.00,9247.00,9689.00,1.00,4,0,2023-03-05 21:14:38.007,9.25,9.69,3.97,1.01,3.99,1.03,0.30,0.70,potok_1,2023-03-05 21:14:38.007,2023,3,9,5,21,14,38,7000,7,Sunday,March,0


## Puste ID_Document

In [68]:
select_14 = pd.read_sql(
"""
    SELECT
         *
    FROM
        data
    WHERE
        ID_Document IS NULL
    

""",
conn
)

In [69]:
select_14

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,629,139824,Potok_0-0,None,22927.00,33396.00,1.00,0,0,2023-02-09 14:21:32.197000,22.93,33.40,4.36,1.38,4.52,1.54,0.30,0.00,None,2023-02-09 14:21:32.197000,2023,2,6,9,14,21,32,197000,4,Thursday,February,1
1,654,676935,Potok_0-0,None,3544.00,4264.00,1.00,0,0,2023-02-02 12:20:18.913000,3.54,4.26,3.55,0.66,3.63,0.72,0.30,0.00,None,2023-02-02 12:20:18.913000,2023,2,5,2,12,20,18,913000,4,Thursday,February,1
2,661,290547,Potok_0-0,None,1217.00,1605.00,1.00,0,0,2023-02-11 21:55:53.220000,1.22,1.60,3.09,0.35,3.21,0.42,0.30,0.00,None,2023-02-11 21:55:53.220000,2023,2,6,11,21,55,53,220000,6,Saturday,February,0
3,719,703355,Potok_0-0,None,82.00,401.00,1.00,0,0,2023-02-02 14:15:14.393000,0.08,0.40,1.92,0.03,2.60,0.15,0.30,0.00,None,2023-02-02 14:15:14.393000,2023,2,5,2,14,15,14,393000,4,Thursday,February,1
4,729,128733,Potok_0-0,None,94.00,763.00,1.00,0,0,2023-02-16 11:13:53.187000,0.09,0.76,1.98,0.04,2.88,0.25,0.30,0.00,None,2023-02-16 11:13:53.187000,2023,2,7,16,11,13,53,187000,4,Thursday,February,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13361,1120317,2216909,Potok_0-0,None,124.00,482.00,1.00,0,0,2023-02-23 11:28:11.813000,0.12,0.48,2.10,0.05,2.68,0.17,0.30,0.00,None,2023-02-23 11:28:11.813000,2023,2,8,23,11,28,11,813000,4,Thursday,February,1
13362,1120373,90238,Potok_0-0,None,3358.00,4068.00,1.00,0,0,2023-02-09 08:18:52.930000,3.36,4.07,3.53,0.64,3.61,0.70,0.30,0.00,None,2023-02-09 08:18:52.930000,2023,2,6,9,8,18,52,930000,4,Thursday,February,0
13363,1120651,24011,Potok_0-0,None,2643.00,3592.00,1.00,0,0,2023-02-12 17:01:16.860000,2.64,3.59,3.42,0.56,3.56,0.66,0.30,0.00,None,2023-02-12 17:01:16.860000,2023,2,6,12,17,1,16,860000,7,Sunday,February,0
13364,1120889,2547349,Potok_0-0,None,85.00,480.00,1.00,0,0,2023-03-10 08:55:46.500000,0.09,0.48,1.93,0.04,2.68,0.17,0.30,0.00,None,2023-03-10 08:55:46.500000,2023,3,10,10,8,55,46,500000,5,Friday,March,0


## 6 lutego wystartowal nowy potok 3-2 ??

In [70]:
select_14 = pd.read_sql(
"""
    SELECT
         *
    FROM
        data
    WHERE
        MessagesInQueue > 1000
    ORDER BY 
        StartDateTimeUtc
    

""",
conn
)

In [71]:
select_14

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,142686,540342,potok_3-2,442509.00,3293.00,4770029.00,1.00,1001,0,2023-02-06 13:52:43.147000,3.29,4770.03,3.52,0.63,6.68,3.68,0.30,3.00,potok_3,2023-02-06 13:52:43.147000,2023,2,6,6,13,52,43,147000,1,Monday,February,1
1,598141,516653,potok_3-2,450487.00,9748.00,4345005.00,3.00,1001,0,2023-02-06 13:52:43.877000,9.75,4345.01,3.99,1.03,6.64,3.64,0.60,3.00,potok_3,2023-02-06 13:52:43.877000,2023,2,6,6,13,52,43,877000,1,Monday,February,1
2,906330,583497,potok_3-2,350427.00,10699.00,5212982.00,1.00,1002,0,2023-02-06 13:52:44.697000,10.70,5212.98,4.03,1.07,6.72,3.72,0.30,3.00,potok_3,2023-02-06 13:52:44.697000,2023,2,6,6,13,52,44,697000,1,Monday,February,1
3,90254,606878,potok_3-2,590471.00,478.00,4764320.00,1.00,1003,0,2023-02-06 13:52:45.767000,0.48,4764.32,2.68,0.17,6.68,3.68,0.30,3.00,potok_3,2023-02-06 13:52:45.767000,2023,2,6,6,13,52,45,767000,1,Monday,February,1
4,214487,509716,potok_3-2,660027.00,NaN,NaN,NaN,1005,0,2023-02-06 13:52:59.757000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,potok_3,2023-02-06 13:52:59.757000,2023,2,6,6,13,52,59,757000,1,Monday,February,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,700911,759513,potok_3-2,765907.00,7364.00,2962137.00,1.00,1109,0,2023-02-06 15:08:51.240000,7.36,2962.14,3.87,0.92,6.47,3.47,0.30,3.05,potok_3,2023-02-06 15:08:51.240000,2023,2,6,6,15,8,51,240000,1,Monday,February,1
1059,461598,506049,potok_3-2,607309.00,5910.00,3290883.00,1.00,1110,0,2023-02-06 15:08:51.373000,5.91,3290.88,3.77,0.84,6.52,3.52,0.30,3.05,potok_3,2023-02-06 15:08:51.373000,2023,2,6,6,15,8,51,373000,1,Monday,February,1
1060,797540,788668,potok_3-2,625574.00,7738.00,3255673.00,1.00,1111,0,2023-02-06 15:08:53.670000,7.74,3255.67,3.89,0.94,6.51,3.51,0.30,3.05,potok_3,2023-02-06 15:08:53.670000,2023,2,6,6,15,8,53,670000,1,Monday,February,1
1061,665551,544641,potok_3-2,554290.00,6539.00,3841956.00,1.00,1112,0,2023-02-06 15:08:58.047000,6.54,3841.96,3.82,0.88,6.58,3.58,0.30,3.05,potok_3,2023-02-06 15:08:58.047000,2023,2,6,6,15,8,58,47000,1,Monday,February,1


## Duration VS MessagesInQueue

In [72]:
select_15 = pd.read_sql(
"""
    SELECT
         ID_Document
         ,COUNT(ID_Process) ID_Process_COUNT
    FROM
        data
    GROUP BY
         ID_Document
    ORDER BY
        ID_Process_COUNT DESC
    
""",
conn
)

In [73]:
select_15

,ID_Document,ID_Process_COUNT
0,NaN,13366
1,38968.00,7
2,36016.00,7
3,33645.00,7
4,31664.00,7
...,...,...
1107614,7.00,1
1107615,6.00,1
1107616,5.00,1
1107617,4.00,1


### ID_Process - Unique

## Czy jest korelacja miedzy Duration a MessagingInQueue

In [74]:
select_16 = pd.read_sql(
"""
    SELECT
         ID_Document
         --,ID_Process
         ,COUNT(ID_Process) ID_Process_COUNT
         ,MessagesInQueue
         ,SUM(Duration) AS Duration_SUM
    FROM
        data
    GROUP BY
        ID_Document,
        --ID_Process,
         MessagesInQueue
    ORDER BY
        MessagesInQueue DESC

""",
conn
)

In [75]:
select_16

,ID_Document,ID_Process_COUNT,MessagesInQueue,Duration_SUM
0,805750.00,1,2018,3323154.00
1,566773.00,1,2017,NaN
2,350649.00,1,2016,4330253.00
3,779489.00,1,2015,NaN
4,367155.00,1,2011,4991851.00
...,...,...,...,...
1107704,2861827.00,1,0,642.00
1107705,2861831.00,1,0,6664.00
1107706,2861833.00,1,0,5648.00
1107707,2861837.00,1,0,4526.00


In [76]:
profile = ProfileReport(select_16, tsmode=False, title="Pandas Profiling Report")

In [77]:
profile.to_file("outputs/report_Duration_SUM.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Mozna odrzucic Potok_0-0 -- brak ID_Document

In [80]:
select_17 = pd.read_sql(
"""
    SELECT DISTINCT
         Queue 
    FROM
        data
    WHERE
        ID_Document IS NULL

""",
conn
)

In [81]:
select_17

,Queue
0,Potok_0-0


In [84]:
select_18 = pd.read_sql(
"""
    SELECT *
    FROM
        data
    WHERE
        ID_Document IS NULL

""",
conn
)

In [85]:
select_18

,index,ID_Process,Queue,ID_Document,ProcessingDuration,Duration,InputPageCount,MessagesInQueue,Retries,StartDateTimeUtc,ProcessingDuration_sec,Duration_sec,Log_ProcessingDuration,Log_ProcessingDuration_sec,Log_Duration,Log_Duration_sec,Log_InputPageCount,Log_MessagesInQueue,potok,date_time,year,month,week,day,hour,minute,second,microsecond,weekday,weekday_name,month_name,working_hours
0,629,139824,Potok_0-0,None,22927.00,33396.00,1.00,0,0,2023-02-09 14:21:32.197000,22.93,33.40,4.36,1.38,4.52,1.54,0.30,0.00,None,2023-02-09 14:21:32.197000,2023,2,6,9,14,21,32,197000,4,Thursday,February,1
1,654,676935,Potok_0-0,None,3544.00,4264.00,1.00,0,0,2023-02-02 12:20:18.913000,3.54,4.26,3.55,0.66,3.63,0.72,0.30,0.00,None,2023-02-02 12:20:18.913000,2023,2,5,2,12,20,18,913000,4,Thursday,February,1
2,661,290547,Potok_0-0,None,1217.00,1605.00,1.00,0,0,2023-02-11 21:55:53.220000,1.22,1.60,3.09,0.35,3.21,0.42,0.30,0.00,None,2023-02-11 21:55:53.220000,2023,2,6,11,21,55,53,220000,6,Saturday,February,0
3,719,703355,Potok_0-0,None,82.00,401.00,1.00,0,0,2023-02-02 14:15:14.393000,0.08,0.40,1.92,0.03,2.60,0.15,0.30,0.00,None,2023-02-02 14:15:14.393000,2023,2,5,2,14,15,14,393000,4,Thursday,February,1
4,729,128733,Potok_0-0,None,94.00,763.00,1.00,0,0,2023-02-16 11:13:53.187000,0.09,0.76,1.98,0.04,2.88,0.25,0.30,0.00,None,2023-02-16 11:13:53.187000,2023,2,7,16,11,13,53,187000,4,Thursday,February,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13361,1120317,2216909,Potok_0-0,None,124.00,482.00,1.00,0,0,2023-02-23 11:28:11.813000,0.12,0.48,2.10,0.05,2.68,0.17,0.30,0.00,None,2023-02-23 11:28:11.813000,2023,2,8,23,11,28,11,813000,4,Thursday,February,1
13362,1120373,90238,Potok_0-0,None,3358.00,4068.00,1.00,0,0,2023-02-09 08:18:52.930000,3.36,4.07,3.53,0.64,3.61,0.70,0.30,0.00,None,2023-02-09 08:18:52.930000,2023,2,6,9,8,18,52,930000,4,Thursday,February,0
13363,1120651,24011,Potok_0-0,None,2643.00,3592.00,1.00,0,0,2023-02-12 17:01:16.860000,2.64,3.59,3.42,0.56,3.56,0.66,0.30,0.00,None,2023-02-12 17:01:16.860000,2023,2,6,12,17,1,16,860000,7,Sunday,February,0
13364,1120889,2547349,Potok_0-0,None,85.00,480.00,1.00,0,0,2023-03-10 08:55:46.500000,0.09,0.48,1.93,0.04,2.68,0.17,0.30,0.00,None,2023-03-10 08:55:46.500000,2023,3,10,10,8,55,46,500000,5,Friday,March,0


In [88]:
select_19 = pd.read_sql(
"""
    SELECT
        Queue
        ,COUNT(Queue) AS Queue_COUNT
    FROM
        data
    GROUP BY
        Queue

""",
conn
)

In [89]:
select_19

,Queue,Queue_COUNT
0,Potok_0-0,13366
1,potok_1-1,271952
2,potok_1-2,333292
3,potok_2-1,1193
4,potok_2-2,497
5,potok_3-1,255158
6,potok_3-2,245732


In [91]:
select_20 = pd.read_sql(
"""
    SELECT
        Queue
        ,COUNT(Queue) AS Queue_COUNT
    FROM
        data
    WHERE
        Queue IN ('potok_2-1', 'potok_2-2')
    GROUP BY
        Queue

""",
conn
)

In [92]:
select_20

,Queue,Queue_COUNT
0,potok_2-1,1193
1,potok_2-2,497
